In [8]:
import pickle
import os
import gzip
import json
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix, accuracy_score
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import GridSearchCV
from sklearn import pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


### **Paso 1: Realice la limpieza de los datasets:**
- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".

Renombre la columna "default payment next month" a "default" y remueva la columna "ID".

In [4]:
# 1.1 Cargar los datos
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")

# 1.2 Organizar los datasets:

def limpiar(df):
    df = df.rename(columns={'default payment next month': 'default'})
    df.drop('ID', axis=1, inplace=True)
    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 4 if x>4 else x)
    df = df.query('MARRIAGE != 0 and EDUCATION != 0')
    df = df.dropna()
    return df


train_data = limpiar(train_data)
test_data = limpiar(test_data)

### **Paso 2: Divida los datasets en x_train, y_train, x_test, y_test.**

In [5]:
x_train = train_data.drop(columns=["default"])
y_train = train_data["default"]

x_test = test_data.drop(columns=["default"])
y_test = test_data["default"]

### **Paso 3: Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:**
- Transforma las variables categoricas usando el método one-hot-encoding.
- Escala las demas variables al intervalo [0, 1].
- Selecciona las K mejores caracteristicas.
- Ajusta un modelo de regresion logistica.

In [17]:
colc = ['SEX','EDUCATION','MARRIAGE']
numc= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]

# Preprocesamiento: Transoformar y Escalar
pre = ColumnTransformer(
    transformers = [
        ("ohe", OneHotEncoder(dtype=int), colc),
        ("scaler",MinMaxScaler(),numc)
    ],
    remainder = "passthrough"
)

# Pipeline
pipeline = Pipeline(steps=[
    ('transformer', pre),
    ('feature_selector', SelectKBest(f_classif, k=5)), # Se empieza con 5 vecinos
    ('logisticregression',  LogisticRegression(max_iter=1000, random_state=2024, n_jobs=-1))
])

pipeline

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(dtype=<class 'int'>),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE']),
                                                 ('scaler', MinMaxScaler(),
                                                  ['LIMIT_BAL', 'AGE', 'PAY_0',
                                                   'PAY_2', 'PAY_3', 'PAY_4',
                                                   'PAY_5', 'PAY_6',
                                                   'BILL_AMT1', 'BILL_AMT2',
                                                   'BILL_AMT3', 'BILL_AMT4',
                                                   'BILL_AMT5', 'BILL_AMT6',
                                                   'PAY_AMT1', 'PAY_AMT2',
                                                   'PAY_AMT3', 'PAY_AMT4',
                                                   'PAY_AMT5', 'PAY_AMT6'])])),
                ('feature_selector', SelectKBest(k=5)),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, n_jobs=-1,
                                    random_state=2024))])

### **Paso 4: Optimice los hiperparametros del pipeline usando validación cruzada. Use 10 splits para la validación cruzada. Use la función de precision balanceada para medir la precisión del modelo.**

In [18]:
param_grid = {
    'logisticregression__C': [1],
    'logisticregression__solver': ['lbfgs'],
    'feature_selector__k': [1],
    'feature_selector__score_func': [f_classif]
}


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=-1,
    refit=True,
    verbose=True
)

grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(dtype=<class 'int'>),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('scaler',
                                                                         MinMaxScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT...
                                       ('feature_selector', SelectKBest(k=5)),
                                       ('logisticregression',
                                        LogisticRegression(max_iter=1000,
                                                           n_jobs=-1,
                                                           random_state=2024))]),
             n_jobs=-1,
             param_grid={'feature_selector__k': [1],
                         'feature_selector__score_func': [<function f_classif at 0x0000020E1B56EF80>],
                         'logisticregression__C': [1],
                         'logisticregression__solver': ['lbfgs']},
             scoring='balanced_accuracy', verbose=True)

### **Paso 5. Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz". Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.**

In [19]:
os.makedirs('../files/models', exist_ok=True)

with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

### **Paso 6: Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba**

In [20]:
def cargar_modelo_y_predecir(data, modelo_path="../files/models/model.pkl.gz"):
    """
    Carga un modelo guardado en un archivo comprimido y realiza predicciones sobre los datos proporcionados.
    
    Args:
        data (array-like): Datos sobre los que se realizarán las predicciones.
        modelo_path (str): Ruta al archivo comprimido que contiene el modelo guardado.

    Returns:
        array-like: Predicciones realizadas por el modelo.
    """
    try:
        with gzip.open(modelo_path, "rb") as file:
            estimator = pickle.load(file)
        return estimator.predict(data)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo de modelo en la ruta especificada: {modelo_path}")
    except Exception as e:
        raise RuntimeError(f"Error al cargar el modelo o realizar predicciones: {e}")

# Uso de la función
y_train_pred = cargar_modelo_y_predecir(x_train)
y_test_pred = cargar_modelo_y_predecir(x_test)

In [21]:
def escribir_metricas(dict_metricas):
    models_dir = '../files/output'
    os.makedirs(models_dir, exist_ok=True)
    
    if os.path.exists('../files/output/metrics.json'):
        with open('../files/output/metrics.json', mode='r') as file:
            if len(file.readlines()) >= 4:
                os.remove('../files/output/metrics.json')
    
    with open('../files/output/metrics.json', mode='a') as file:
        file.write(str(dict_metricas).replace("'",'"')+"\n")

        
def eval_metrics(dataset,y_true, y_pred):
    accuracy = float(accuracy_score(y_true, y_pred))
    precision= float(precision_score(y_true, y_pred))
    balanced_accuracy = float(balanced_accuracy_score(y_true, y_pred))
    recall = float(recall_score(y_true, y_pred))
    f1 = float(f1_score(y_true, y_pred))
    metrics= {
        "type": "metrics",
        "dataset": dataset,
        "precision": precision,
        "balanced_accuracy": balanced_accuracy,
        "recall": recall,
        "f1_score": f1
    }
    
    escribir_metricas(metrics)

metrics_train = eval_metrics('train',y_train,y_train_pred)
metrics_test = eval_metrics('test',y_test,y_test_pred)

### **Paso 7: Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba.**

In [22]:
def matriz_confusion(dataset,y_true, y_pred):
    
    matriz = confusion_matrix(y_true, y_pred)
    
    matrix_confusion = {
        "type":"cm_matrix",
        "dataset": dataset,
        "true_0":{
            "predicted_0": int(matriz[0,0]),
            "predicted_1": None
        },
        "true_1":{
            "predicted_0": None,
            "predicted_1": int(matriz[1,1])
        }  
    }
    
    escribir_metricas(json.dumps(matrix_confusion))


metrics_train = matriz_confusion('train',y_train,y_train_pred)
metrics_test = matriz_confusion('test',y_test,y_test_pred)